In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('supervised_learning_data/40_lightbulbs_16_goals_50k_trajectories_20250819-171433.csv', index_col= False )

In [48]:
print(df.shape)
df.head()

(552928, 6)


,rnn_hidden_state,bayes_posterior,robot_action,human_action,trajectory_id,time_step
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.062...",15,37,0,0
1,"[[0.8756062984466553, 0.03497914969921112, -0....","[0.125, 0.125, 0.125, 0.125, 0.0, 0.0, 0.0, 0....",17,13,0,1
2,"[[0.8377193212509155, -0.029644737020134926, 0...","[0.125, 0.125, 0.125, 0.125, 0.0, 0.0, 0.0, 0....",29,15,0,2
3,"[[0.799655556678772, -0.050025902688503265, 0....","[0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0...",31,33,0,3
4,"[[0.7611371278762817, -0.07170260697603226, 0....","[0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0...",21,14,0,4


In [45]:
# hidden size
len(df.iloc[0]['rnn_hidden_state'][0])

64

In [8]:
lens = df.groupby('trajectory_id').size()
print(lens.describe())

count    50000.000000
mean        11.058560
std          1.759054
min          4.000000
25%         10.000000
50%         11.000000
75%         12.000000
max         22.000000
dtype: float64


In [9]:
# 2) Recompute via max(time_step) + 1 and compare
lens1 = df.groupby('trajectory_id').size()
lens2 = df.groupby('trajectory_id')['time_step'].max().add(1)
float(lens1.mean()), float(lens2.mean()), (lens1 == lens2).all()

(11.05856, 11.05856, np.True_)

In [52]:
import ast
import numpy as np
import pandas as pd

def coerce_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert stringified list columns and ensure ints where needed."""
    df = df.copy()
    for col in ["rnn_hidden_state", "bayes_posterior"]:
        df[col] = df[col].apply(ast.literal_eval)
    df["trajectory_id"] = df["trajectory_id"].astype(int)
    df["time_step"] = df["time_step"].astype(int)
    return df

def posterior_stats(df: pd.DataFrame) -> pd.DataFrame:
    """Compute trajectory-level stats on bayes_posterior."""
    def _is_one_hot(a):
        a = np.asarray(a, dtype=float)
        return (a >= 0).all() and np.isclose(a.sum(), 1.0) and (np.count_nonzero(a) == 1)

    onehot = df["bayes_posterior"].apply(_is_one_hot)
    pos = df.groupby("trajectory_id").cumcount()
    traj_len = df.groupby("trajectory_id").size()

    first_onehot_pos = pos.where(onehot).groupby(df["trajectory_id"]).min()
    steps_until = first_onehot_pos.reindex(traj_len.index).fillna(traj_len).astype(int)

    change = df["bayes_posterior"].ne(
        df.groupby("trajectory_id")["bayes_posterior"].shift()
    )
    block = change.groupby(df["trajectory_id"]).cumsum()
    longest_streak = (
        df.groupby(["trajectory_id", block]).size().groupby(level=0).max()
    ).reindex(traj_len.index).astype(int)

    return pd.DataFrame({
        "trajectory_id": traj_len.index,
        "trajectory_length": traj_len.values,
        "timesteps_until_point_mass_posterior": steps_until.values,
        "longest_streak_without_bayes_update": longest_streak.values,
    }).reset_index(drop=True)


In [ ]:
df = coerce_columns(df)
stats = posterior_stats(df)

In [57]:
display(stats)
stats.describe()
#TODO longest streak before certainty

,trajectory_id,trajectory_length,timesteps_until_point_mass_posterior,longest_streak_without_bayes_update
0,0,13,10,5
1,1,9,6,3
2,2,9,8,4
3,3,10,6,4
4,4,14,7,7
...,...,...,...,...
49995,49995,15,5,10
49996,49996,10,7,4
49997,49997,8,5,3
49998,49998,12,5,7


,trajectory_id,trajectory_length,timesteps_until_point_mass_posterior,longest_streak_without_bayes_update
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,24999.500000,11.058560,6.277640,5.409200
std,14433.901067,1.759054,1.726064,1.736633
min,0.000000,4.000000,4.000000,1.000000
25%,12499.750000,10.000000,5.000000,4.000000
50%,24999.500000,11.000000,6.000000,5.000000
75%,37499.250000,12.000000,7.000000,7.000000
max,49999.000000,22.000000,15.000000,14.000000


In [12]:
stats[stats['longest_streak_without_bayes_update'] > 13]

,trajectory_id,trajectory_length,timesteps_until_point_mass_posterior,longest_streak_without_bayes_update
16182,16182,22,8,14
23537,23537,18,4,14
32608,32608,18,4,14


In [74]:
df[df['trajectory_id'] == 16182]

,rnn_hidden_state,bayes_posterior,robot_action,human_action,trajectory_id,time_step,human_action_mod4,robot_action_mod4
179333,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.062...",6,2,16182,0,2,2
179334,"[[0.7623766660690308, 0.03923240303993225, 0.0...","[0.0, 0.0, 0.125, 0.125, 0.0, 0.0, 0.125, 0.12...",6,1,16182,1,1,2
179335,"[[0.5139753222465515, -0.33282145857810974, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0...",9,8,16182,2,0,1
179336,"[[0.32383859157562256, 0.1321951299905777, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12,10,16182,3,2,0
179337,"[[0.2847595810890198, 0.08674371242523193, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",29,24,16182,4,0,1
179338,"[[0.2696472108364105, 0.04116964712738991, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",19,21,16182,5,1,3
179339,"[[0.2502862513065338, -0.05075768008828163, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",15,37,16182,6,1,3
179340,"[[0.22544290125370026, -0.14941637217998505, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",27,7,16182,7,3,3
179341,"[[0.1938486397266388, -0.2100922167301178, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",7,31,16182,8,3,3
179342,"[[0.1648888885974884, -0.25930920243263245, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2,19,16182,9,3,2


In [59]:
import numpy as np
import pandas as pd
import ast
import jax
import jax.numpy as jnp
import optax

def _to_arr(x): # TODO I'm pretty sure this function is not necessary (x is already not a string in my dataframe), but I don't have time to move stuff around rn. 
    if isinstance(x, str):
        x = ast.literal_eval(x)
    a = np.asarray(x, dtype=np.float32)
    return a.reshape(-1)

def _prepare(df): #TODO also not convinced this function is necessary
    X = np.stack([_to_arr(x) for x in df["rnn_hidden_state"].values])
    Y = np.stack([_to_arr(y) for y in df["bayes_posterior"].values])
    Y = (Y / np.clip(Y.sum(axis=1, keepdims=True), 1e-8, None)).astype(np.float32)
    return X, Y 

def _split_by_traj(df, test_frac=0.2, seed=0):
    '''
    Splits trajectories in test and train set.
    it's important that this is done by trajectory_id,
    If we just split the rows of the original dataframe we would 
    have rows corresponding to different timesteps of the same trajectory in train and test and that's a data leak I think. 
    '''
    ids = df["trajectory_id"].unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(ids)
    n_test = max(1, int(round(len(ids) * test_frac)))
    test_ids = set(ids[:n_test]) # this shuffle thing and selecting the first n seems like a roundabout way to sample but i guess it works
    train_df = df[~df["trajectory_id"].isin(test_ids)]
    test_df = df[df["trajectory_id"].isin(test_ids)]
    return train_df, test_df, sorted(test_ids)

def _train_probe(Xtr, Ytr, Xte, Yte, lr=1e-2, weight_decay=1e-4, epochs=50, batch_size=8192, seed=0):
    H, K = Xtr.shape[1], Ytr.shape[1] # K ends up being the number of possible goals 
    key = jax.random.PRNGKey(seed)
    params = (
        jax.random.normal(key, (H, K)) * 0.01,
        jnp.zeros((K,), dtype=jnp.float32),
    )
    opt = optax.adamw(learning_rate=lr, weight_decay=weight_decay)
    opt_state = opt.init(params)

    def predict(p, X):
        W, b = p
        return jax.nn.softmax(X @ W + b, axis=-1)

    def loss_fn(p, X, Y):
        P = predict(p, X)
        ce = -jnp.sum(Y * jnp.log(P + 1e-9), axis=1).mean() #TODO this 1e-9 is a chatGPT thing, I'm not so convinced this is a good way to handle when things get too close to 0...
        # try googling "log sum exp trick" 
        # or "how can i make this ce more stable"
        return ce

    @jax.jit # the decorator is basically doing the reassignment step = jax.jit(step) 
    def step(p, s, X, Y):
        (loss_val, grads) = jax.value_and_grad(loss_fn)(p, X, Y)
        updates, s = opt.update(grads, s, p)
        p = optax.apply_updates(p, updates)
        return p, s, loss_val

    Xtr_j, Ytr_j = jnp.asarray(Xtr), jnp.asarray(Ytr)
    n = Xtr.shape[0]
    idx = np.arange(n)

    for e in range(epochs):
        rng = np.random.default_rng(seed + e)
        rng.shuffle(idx)
        for i in range(0, n, batch_size):
            j = idx[i : i + batch_size]
            params, opt_state, _ = step(params, opt_state, Xtr_j[j], Ytr_j[j])

    def metrics(X, Y):
        P = predict(params, jnp.asarray(X))
        ce = float((-jnp.sum(jnp.asarray(Y) * jnp.log(P + 1e-9), axis=1)).mean())
        acc = float((jnp.argmax(P, axis=1) == np.argmax(Y, axis=1)).mean()) # TODO this metric does not make sense for us 
        # it's saying on average how often does the probes most likely class coincide with ground truths most likely class
        # but that doesn't make a lot of sense for uniform distirbutions. 
        return ce, acc

    train_ce, train_acc = metrics(Xtr, Ytr)
    test_ce, test_acc = metrics(Xte, Yte)
    return params, dict(train_ce=train_ce, train_acc=train_acc, test_ce=test_ce, test_acc=test_acc)

def train_linear_probe(df, test_frac=0.2, seed=0, lr=1e-2, weight_decay=1e-4, epochs=50, batch_size=8192):
    train_df, test_df, heldout_ids = _split_by_traj(df, test_frac=test_frac, seed=seed)
    Xtr, Ytr = _prepare(train_df)
    Xte, Yte = _prepare(test_df)

    mean = Xtr.mean(axis=0, keepdims=True).astype(np.float32)
    std = (Xtr.std(axis=0, keepdims=True) + 1e-6).astype(np.float32)
    Xtr = (Xtr - mean) / std
    Xte = (Xte - mean) / std

    params, metrics = _train_probe(
        Xtr, Ytr, Xte, Yte,
        lr=lr, weight_decay=weight_decay, epochs=epochs, batch_size=batch_size, seed=seed
    )

    probe = {"W": np.array(params[0]), "b": np.array(params[1]), "mean": mean[0], "std": std[0], "classes": Ytr.shape[1]}
    return probe, metrics, heldout_ids, Yte

def apply_probe(probe, hidden_state):
    x = np.asarray(hidden_state, dtype=np.float32).reshape(-1)
    x = (x - probe["mean"]) / probe["std"]
    logits = x @ probe["W"] + probe["b"]
    logits = logits - logits.max()
    p = np.exp(logits)
    p /= p.sum()
    return p

def _eval_on_df_with_probe(probe, df):
    if len(df) == 0:
        return {"n": 0, "ce": float("nan"), "acc": float("nan")}
    X, Y = _prepare(df)
    X = (X - probe["mean"]) / probe["std"]
    logits = X @ probe["W"] + probe["b"]
    logits = logits - logits.max(axis=1, keepdims=True)
    P = np.exp(logits); P /= P.sum(axis=1, keepdims=True)
    ce = float((-np.sum(Y * np.log(P + 1e-9), axis=1)).mean())
    acc = float((np.argmax(P, axis=1) == np.argmax(Y, axis=1)).mean())
    return {"n": int(len(df)), "ce": ce, "acc": acc}

def test_time_range(df, probe, heldout_ids, tmin=None, tmax=None):
    test_df = df[df["trajectory_id"].isin(heldout_ids)]
    if tmin is not None:
        test_df = test_df[test_df["time_step"] >= tmin]
    if tmax is not None:
        test_df = test_df[test_df["time_step"] <= tmax]
    return _eval_on_df_with_probe(probe, test_df)


probe, metrics, heldout, Yte = train_linear_probe(df, test_frac=0.2, seed=2024, lr=5e-3, epochs=100, batch_size=16384)
print(metrics)
# p = apply_probe(probe, df.iloc[0]["rnn_hidden_state"])

test_time_range(df, probe, heldout, tmin = 10)

# TODO accuracy doesn't make sense for this use case.
# TODO baseline?? How to know if 1.2 is good or not? (uniform predictor below)


{'train_ce': 1.20596182346344, 'train_acc': 0.6084592342376709, 'test_ce': 1.20656156539917, 'test_acc': 0.6093438863754272}


{'n': 13208, 'ce': 0.08527214080095291, 'acc': 0.9826620230163537}

In [60]:
def test_uniform_predictor(Ytest):
    '''
    For a baseline. 
    Always predict uniform distribution and compare with ground truth. 
    '''
    N, K = Ytest.shape
    U = np.ones((N, K), dtype=np.float32) / K
    ce = float((-np.sum(Ytest * np.log(U + 1e-9), axis=1)).mean())
    acc = float((np.argmax(U, axis=1) == np.argmax(Ytest, axis=1)).mean())
    return dict(ce=ce, acc=acc)

test_uniform_predictor(Yte)

{'ce': 2.7725892066955566, 'acc': 0.23520742160656805}

In [61]:
# what does _prepare do?

A, B = _prepare(df)
A.shape, B.shape, df.shape

# ((552928, 64), (552928, 16), (552928, 6))
# The shape of A makes sense because it is the hidden rnn states:
# each one has 64 values (the hidden size is 64 for this model)
# and the dataframe has 552928 rows (total time steps from all trajectories)
# The shape of B also make sense because it is the posterior distributions
# Each one has 16 values. 

((552928, 64), (552928, 16), (552928, 6))

In [50]:
def show_random_test_trajectories(df, probe, heldout_ids, num_traj=10, seed=0, decimals=2, max_steps=None):
    test_df = df[df["trajectory_id"].isin(heldout_ids)]
    traj_ids = test_df["trajectory_id"].unique()
    if len(traj_ids) == 0:
        print("No held-out trajectories."); return
    rng = np.random.default_rng(seed)
    rng.shuffle(traj_ids)
    pick = traj_ids[: min(num_traj, len(traj_ids))]

    for tid in pick:
        sub = test_df[test_df["trajectory_id"] == tid].sort_values("time_step")
        if max_steps is not None:
            sub = sub.head(max_steps)

        X = np.stack([_to_arr(x) for x in sub["rnn_hidden_state"].values]).astype(np.float32)
        X = (X - probe["mean"]) / probe["std"]
        logits = X @ probe["W"] + probe["b"]
        logits = logits - logits.max(axis=1, keepdims=True)
        P = np.exp(logits); P /= P.sum(axis=1, keepdims=True)

        Y = np.stack([_to_arr(y) for y in sub["bayes_posterior"].values]).astype(np.float32)
        Y = Y / np.clip(Y.sum(axis=1, keepdims=True), 1e-8, None)

        # format into plain python floats
        def fmt(arr):
            return [f"{v:.{decimals}f}" for v in arr]

        Y_fmt = [fmt(row) for row in Y]
        P_fmt = [fmt(row) for row in P]

        out = pd.DataFrame({
            "t": sub["time_step"].values,
            "Y_true": Y_fmt,
            "P_pred": P_fmt,
        })

        print(f"\ntrajectory_id={tid} (steps={len(sub)})")

        display(out)

show_random_test_trajectories(df, probe, heldout, 10, 2024, 2, 100)

#TODO fix off by one here


trajectory_id=27575 (steps=12)


,t,Y_true,P_pred
0,0,"[0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[0.07, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0..."
1,1,"[0.12, 0.12, 0.12, 0.12, 0.00, 0.00, 0.00, 0.0...","[0.06, 0.04, 0.08, 0.07, 0.06, 0.04, 0.08, 0.0..."
2,2,"[0.00, 0.00, 0.25, 0.25, 0.00, 0.00, 0.00, 0.0...","[0.11, 0.06, 0.14, 0.12, 0.00, 0.00, 0.00, 0.0..."
3,3,"[0.00, 0.00, 0.25, 0.25, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.38, 0.12, 0.00, 0.00, 0.00, 0.0..."
4,4,"[0.00, 0.00, 0.50, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.39, 0.20, 0.00, 0.00, 0.01, 0.0..."
5,5,"[0.00, 0.00, 0.50, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.61, 0.01, 0.00, 0.00, 0.01, 0.0..."
6,6,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.66, 0.01, 0.00, 0.00, 0.00, 0.0..."
7,7,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.03, 0.00, 0.00, 0.00, 0.00, 0.0..."
8,8,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.0..."
9,9,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0..."



trajectory_id=49318 (steps=13)


,t,Y_true,P_pred
0,0,"[0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[0.07, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0..."
1,1,"[0.12, 0.00, 0.12, 0.00, 0.12, 0.00, 0.12, 0.0...","[0.08, 0.05, 0.09, 0.07, 0.08, 0.04, 0.08, 0.0..."
2,2,"[0.12, 0.00, 0.12, 0.00, 0.12, 0.00, 0.12, 0.0...","[0.12, 0.01, 0.13, 0.01, 0.11, 0.02, 0.16, 0.0..."
3,3,"[0.12, 0.00, 0.12, 0.00, 0.12, 0.00, 0.12, 0.0...","[0.10, 0.00, 0.14, 0.00, 0.11, 0.00, 0.19, 0.0..."
4,4,"[0.25, 0.00, 0.25, 0.00, 0.25, 0.00, 0.25, 0.0...","[0.06, 0.00, 0.12, 0.00, 0.06, 0.00, 0.14, 0.0..."
5,5,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.10, 0.00, 0.37, 0.00, 0.10, 0.00, 0.42, 0.0..."
6,6,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.29, 0.01, 0.70, 0.0..."
7,7,"[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.32, 0.01, 0.67, 0.0..."
8,8,"[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.0...","[0.01, 0.00, 0.00, 0.00, 0.94, 0.02, 0.02, 0.0..."
9,9,"[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.0...","[0.01, 0.00, 0.00, 0.00, 0.96, 0.02, 0.01, 0.0..."



trajectory_id=1549 (steps=11)


...

,t,Y_true,P_pred
0,0,"[0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[0.07, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0..."
1,1,"[0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.1...","[0.03, 0.11, 0.02, 0.05, 0.03, 0.10, 0.02, 0.0..."
2,2,"[0.25, 0.00, 0.25, 0.00, 0.25, 0.00, 0.25, 0.0...","[0.07, 0.06, 0.30, 0.12, 0.05, 0.06, 0.24, 0.1..."
3,3,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.13, 0.00, 0.31, 0.01, 0.13, 0.00, 0.40, 0.0..."
4,4,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.26, 0.02, 0.69, 0.0..."
5,5,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.00, 0.00, 0.01, 0.00, 0.30, 0.03, 0.66, 0.0..."
6,6,"[0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.50, 0.0...","[0.00, 0.00, 0.01, 0.00, 0.20, 0.02, 0.75, 0.0..."



trajectory_id=13685 (steps=9)


,t,Y_true,P_pred
0,0,"[0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[0.07, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0..."
1,1,"[0.00, 0.12, 0.00, 0.12, 0.00, 0.12, 0.00, 0.1...","[0.04, 0.05, 0.04, 0.07, 0.05, 0.07, 0.06, 0.0..."
2,2,"[0.00, 0.00, 0.00, 0.25, 0.00, 0.00, 0.00, 0.2...","[0.00, 0.20, 0.00, 0.19, 0.00, 0.07, 0.00, 0.0..."
3,3,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.5...","[0.00, 0.01, 0.00, 0.31, 0.00, 0.01, 0.01, 0.2..."
4,4,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.5...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.01, 0.5..."
5,5,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.4..."
6,6,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.02, 0.9..."
7,7,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.9..."
8,8,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.0...","[0.00, 0.00, 0.00, 0.01, 0.00, 0.01, 0.01, 0.9..."



trajectory_id=25510 (steps=12)


,t,Y_true,P_pred
0,0,"[0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[0.07, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0..."
1,1,"[0.12, 0.12, 0.00, 0.00, 0.12, 0.12, 0.00, 0.0...","[0.05, 0.03, 0.10, 0.06, 0.06, 0.04, 0.12, 0.0..."
2,2,"[0.00, 0.00, 0.00, 0.00, 0.25, 0.25, 0.00, 0.0...","[0.19, 0.09, 0.01, 0.00, 0.14, 0.06, 0.01, 0.0..."
3,3,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.21, 0.23, 0.00, 0.0..."
4,4,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.00, 0.38, 0.01, 0.0..."
5,5,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.01, 0.51, 0.01, 0.0..."
6,6,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.01, 0.54, 0.02, 0.0..."
7,7,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.00, 0.00, 0.00, 0.01, 0.63, 0.02, 0.0..."
8,8,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.50, 0.00, 0.0...","[0.00, 0.01, 0.00, 0.00, 0.01, 0.62, 0.02, 0.0..."
9,9,"[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...","[0.00, 0.02, 0.00, 0.00, 0.01, 0.74, 0.01, 0.0..."
